In [1]:
import os

In [2]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification\\research'

In [3]:
os.chdir('c:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification')

In [4]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/quamrl-hoda/kidneyDiseaseClassificationDeepLearningProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "quamrl-hoda"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "10bd4471835788590657bbf53e006492e901bc34"


In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model(
    "artifacts/training/model.h5",
    compile=False
)


In [8]:
model = tf.keras.models.load_model(
    "artifacts/training/model.h5",
    compile=False
)


In [9]:
# model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories,save_json

In [ ]:
class ConfigurationManager:
  def __init__(
      self,
      config_filepath:Path=CONFIG_FILE_PATH,
      params_filepath:Path=PARAMS_FILE_PATH
  ):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])
  
  def get_evaluation_config(self) -> EvaluationConfig:
    eval_config = EvaluationConfig(
      path_of_model=Path("artifacts/training/model.h5"),
      training_data="C:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification\\artifacts\\data_ingestion\\content\\drive\\MyDrive\\CT-kidney-data",
      mlflow_uri="https://dagshub.com/quamrl-hoda/kidneyDiseaseClassificationDeepLearningProject.mlflow",
      all_params=self.params,
      params_image_size=self.params.IMAGE_SIZE,
      params_batch_size=self.params.BATCH_SIZE
    )
    return eval_config

In [13]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification'

In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [16]:
class Evaluation:
  def __init__(self, config:EvaluationConfig):
    self.config = config

  def _valid_generator(self):

    datagenerator_kwargs = dict(
      rescale=1./255,
      validation_split=0.30 # 30% data for validation
    )
    dataflow_kwargs = dict(
      target_size=self.config.params_image_size[:-1],
      batch_size=self.config.params_batch_size,
      class_mode="categorical"
    )

    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

    self.valid_generator = valid_datagenerator.flow_from_directory(
      directory=self.config.training_data,
      subset="validation",
      shuffle=False,  
      **dataflow_kwargs
    )

  @staticmethod
  def load_model(path:Path) -> tf.keras.Model:
      model = tf.keras.models.load_model(path)
      return model
    
  def evaluation(self):
      self.model = self.load_model(self.config.path_of_model)
      self._valid_generator()
      self.score = self.model.evaluate(self.valid_generator)
      self.save_score()
  # def save_score(self):
  #     scores = {"loass":self.score[0], "accuracy":self.score[1]}
  #     save_json(Path("artifacts/evaluation/scores.json"), data=scores)
  def save_score(self):
    scores = {"loss": self.score[0], "accuracy": self.score[1]}
    # create folder if not exists
    os.makedirs("artifacts/evaluation", exist_ok=True)
    save_json(Path("artifacts/evaluation/scores.json"), data=scores)

  def log_into_mlflow(self):

    # Clean parameters for MLflow
    clean_params = {}
    for key, value in self.config.all_params.items():
        if isinstance(value, (list, tuple, dict)):
            clean_params[key] = str(value)
        else:
            clean_params[key] = value

    mlflow.log_params(clean_params)

    # Log metrics
    mlflow.log_metrics({
        "eval_loss": self.score[0],
        "eval_accuracy": self.score[1]
    })

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        mlflow.keras.log_model(
            self.model,
            "model",
            registered_model_name="VGG16Model"
        )
    else:
        mlflow.keras.log_model(self.model, "model")


In [17]:
# from tensorflow.keras.models import load_model
# model = load_model("C:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification\\artifacts\\prepare_base_model\\base_model_updated.h5")
# model.input_shape


In [18]:
# print("Loaded model summary:")
# model.summary()


In [ ]:
try:
  config = ConfigurationManager()
  eval_config = config.get_evaluation_config()

  evaluation = Evaluation(config=eval_config)
  evaluation.evaluation()
  evaluation.save_score()
  evaluation.log_into_mlflow()
except Exception as e:
  raise e

[2025-12-02 22:57:22,331:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-12-02 22:57:22,335:INFO:common:yaml file: params.yaml loaded successfully]
[2025-12-02 22:57:22,337:INFO:common:created directory at: artifacts]
Found 3735 images belonging to 4 classes.
234/234 [==============================] - 592s 3s/step - loss: 7.2370 - accuracy: 0.5847
[2025-12-02 23:07:15,064:INFO:common:json file saved at: artifacts\evaluation\scores.json]
[2025-12-02 23:07:23,157:WARNING:connectionpool:Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /quamrl-hoda/kidneyDiseaseClassificationDeepLearningProject.mlflow/api/2.0/mlflow/runs/create]


2025/12/02 23:07:34 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-12-02 23:07:36,551:WARNING:save:Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\quamr\AppData\Local\Temp\tmph42pzujt\model\data\model\assets
[2025-12-02 23:07:37,178:INFO:builder_impl:Assets written to: C:\Users\quamr\AppData\Local\Temp\tmph42pzujt\model\data\model\assets]


c:\Users\quamr\OneDrive\Desktop\project\kidneyDiseaseClassification\envmlflow\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2025-12-02 23:10:05,515:WARNING:connectionpool:Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')': /quamrl-hoda/kidneyDiseaseClassificationDeepLearningProject.mlflow/api/2.0/mlflow-artifacts/artifacts/086b8f457c664fc6a41e6bd1a4b2591a/cd19abe29ac94de4a4bdb423bca50256/artifacts/model/data/model/saved_model.pb]
[2025-12-02 23:10:18,353:WARNING:connectionpool:Retrying (Retry(total=3, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')': /quamrl-hoda/kidneyDiseaseClassificationDeepLearningProject.mlflow/api/2.0/mlflow-artifacts/artifacts/086b8f457c664fc6a41e6bd1a4b2591a/cd19abe29ac94de4a4bdb423bca50256/artifacts/model/data/model/saved_model.pb]
[2025-12-02 23:10:46,809:WARNING:connectionpool:Retrying (Retry(total=2, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEO

MlflowException: API request to https://dagshub.com/quamrl-hoda/kidneyDiseaseClassificationDeepLearningProject.mlflow/api/2.0/mlflow-artifacts/artifacts/086b8f457c664fc6a41e6bd1a4b2591a/cd19abe29ac94de4a4bdb423bca50256/artifacts/model/data/model/saved_model.pb failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /quamrl-hoda/kidneyDiseaseClassificationDeepLearningProject.mlflow/api/2.0/mlflow-artifacts/artifacts/086b8f457c664fc6a41e6bd1a4b2591a/cd19abe29ac94de4a4bdb423bca50256/artifacts/model/data/model/saved_model.pb (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')))